Construct a pyomo model based on the following linear program:

$$
\begin{array}{ll} \min & 2 x_1 + 3 x_2\\ \mathrm{s.t.} & 3 x_1 + 4 x_2 \geq 1\\ & x_1, x_2 \geq 0 \end{array}
$$

<!-- $$
\begin{array}{lll} 
\min & \sum_{j=1}^n c_j x_j &\\ \mathrm{s.t.} & \sum_{j=1}^n a_{ij} x_j \geq b_i & \forall i = 1 \ldots m\\ & x_j \geq 0 & \forall j = 1 \ldots n 
\end{array}
$$ -->

# 1. IMPORT

In [1]:
import pyomo as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory

# 2. DEFINE THE MODEL

In [26]:
# define the model
model = ConcreteModel(name='my_first_model')

# step 1: define the decision variables
model.x1 = Var(within=NonNegativeReals)
model.x2 = Var(within=NonNegativeReals)

# step 2: define the constraints
def constraint_rule1(model):
    return 3*model.x1 + 4*model.x2 >= 1
model.cons1 = Constraint(rule=constraint_rule1)

# step 3: define the objective function
def objective_rule(model):
    return 2*model.x1 + 3*model.x2
model.obj = Objective(rule=objective_rule, sense=minimize)

# 3. SOLVE THE MODEL

In [51]:
glpk_res = SolverFactory('glpk').solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 0.666666666666667
  Upper bound: 0.666666666666667
  Number of objectives: 1
  Number of constraints: 1
  Number of variables: 2
  Number of nonzeros: 2
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.012667179107666016
# ----------------------------------------------------------
#   Solution Information
# ------------

In [54]:
SolverFactory('scip').solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: 0.666666666666666
  Number of objectives: 1
  Number of constraints: 0
  Number of variables: 2
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: optimal solution found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.01
  Gap: 0.0
  Primal bound: 0.666666666666666
  Dual bound: 0.666666666666666
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number 

In [55]:
print('obj =', model.obj())
print('x1 =', model.x1())
print('x2 =', model.x2())

obj = 0.6666666666666666
x1 = 0.3333333333333333
x2 = 0.0


# 4. EXPORT MODEL TO MPS/LP FILE

In [29]:
model.write('./test_model.lp', io_options={'symbolic_solver_labels': True})

('./test_model.lp', 4605974560)

In [30]:
model.write('./test_model.mps', io_options={'symbolic_solver_labels': True})

('./test_model.mps', 4605974416)

# 5. IMPORT MPS/LP FILE

## 5.1 OR-TOOLS

In [33]:
from ortools.linear_solver import pywraplp
from ortools.linear_solver.python import model_builder

In [90]:
ortools_model = model_builder.ModelBuilder()
# ortools_model.import_from_lp_file('./test_model.lp')
ortools_model.import_from_mps_file('./test_model.mps')


ortools_solver = model_builder.ModelSolver('SCIP') # GLOP
ortools_solver.solve(ortools_model)

print('objective value =', ortools_solver.objective_value)
ortools_variables = [ortools_model.var_from_index(i) for i in range(ortools_model.num_variables)]
for variable in ortools_variables:
    print('%s = %f' % (variable.name, ortools_solver.value(variable)))

objective value = 0.6666666666860692
x1 = 0.333333
x2 = 0.000000
